In [ ]:
!pip install -q "labelbox[data]"

In [ ]:
from labelbox import Client, OntologyBuilder
from labelbox.data.annotation_types import Geometry
from labelbox.data.annotation_types.collection import LabelList
from PIL import Image
import numpy as np
import os
import torch

In [ ]:
class SegClsName:
  VESSEL = "血管"
  BRONCHUS = "支气管"

  def get_all_names():
    return ['支气管', '血管']

class VesselRatingName:
  D = "D血管周围浸润"

class BronchusRatingName:
  A = "A支气管浸润"
  B = "B支气管浸润定性"
  C = "C支气管腔渗出"

  def get_all_names():
    return ["A支气管浸润", "B支气管浸润定性", "C支气管腔渗出"]

In [ ]:
API_KEY = "your-k"
PROJECT_ID = "yours"
client = Client(api_key=API_KEY)
project = client.get_project(PROJECT_ID)
labels = project.label_generator().as_list()

/usr/local/lib/python3.7/dist-packages/labelbox/data/annotation_types/classification/classification.py:85: UserWarning: Dropdown classification is deprecated and will be removed in a future release
  warnings.warn("Dropdown classification is deprecated and will be "


In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
import cv2
from matplotlib import pyplot as plt

In [ ]:
nemo = np.array(Image.fromarray(labels[1].data.value).convert("RGB"))
# Image.fromarray(nemo[200:300, 100:200]).resize((1000, 1000))

In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

def color_seg(roi, target):
    # roi is the object or region of object we need to find
    hsv = cv.cvtColor(roi, cv.COLOR_BGR2HSV)

    # target is the image we search in
    # target = nemo
    hsvt = cv.cvtColor(target, cv.COLOR_BGR2HSV)

    # calculating object histogram
    # 图像hsv，第一通道和第二通道，没有mask，分成多少份（h范围就是到180，所以现在bin是1）,从0-180的像素
    roihist = cv.calcHist([hsv], [0, 1], None, [180, 256], [0, 180, 0, 256])

    # normalize histogram and apply backprojection
    cv.normalize(roihist, roihist, 0, 255, cv.NORM_MINMAX)
    dst = cv.calcBackProject([hsvt], [0, 1], roihist, [0, 180, 0, 256], 1)

    # Now convolute with circular disc
    disc = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))
    cv.filter2D(dst, -1, disc, dst)

    # threshold and binary AND
    ret, thresh = cv.threshold(dst, 0, 255, 0)
    thresh = cv.merge((thresh, thresh, thresh))
    res = cv.bitwise_and(target, thresh)

    #显示出来源图像，图像 mask 和抠图结果
    res = np.vstack((target, thresh, res))
    cv.imwrite('res.jpg', res)

    area = np.where(thresh[:,:,0])[0].shape[0]
    return roihist, thresh, area

roi_ = cv.imread('/content/res.jpg')
target_ = cv.imread('/content/drive/MyDrive/102-400-1.jpg')
roihist_, th, ar = color_seg(roi_, target_)
np.save('./background.npy', roihist_)
roihist_, th, ar

In [ ]:
def get_cytoplasm_area(img):
  hist = np.load('cytoplasm.npy')
  hsvt = cv.cvtColor(img, cv.COLOR_BGR2HSV)
  dst = cv.calcBackProject([hsvt], [0, 1], hist, [0, 180, 0, 256], 1)

  # Now convolute with circular disc
  disc = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))
  cv.filter2D(dst, -1, disc, dst)

  # threshold and binary AND
  ret, thresh = cv.threshold(dst, 0, 255, 0)
  thresh = cv.merge((thresh, thresh, thresh))

  return np.where(thresh[:,:,0])[0].shape[0] 

In [ ]:
def get_area(img, histfile='nucleus.npy'):
  hist = np.load(histfile)
  hsvt = cv.cvtColor(img, cv.COLOR_BGR2HSV)
  dst = cv.calcBackProject([hsvt], [0, 1], hist, [0, 180, 0, 256], 1)

  # Now convolute with circular disc
  disc = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))
  cv.filter2D(dst, -1, disc, dst)

  # threshold and binary AND
  ret, thresh = cv.threshold(dst, 0, 255, 0)
  thresh = cv.merge((thresh, thresh, thresh))

  return np.where(thresh[:,:,0])[0].shape[0]

def get_nucleus_area(img):
  return get_area(img, 'nucleus.npy')

def get_cytoplasm_area(img):
  return get_area(img, 'cytoplasm.npy')

In [ ]:
test_img = cv.imread('/content/drive/MyDrive/102-400-1.jpg')
get_nucleus_area(test_img)/get_cytoplasm_area(test_img)

0.2083661847939391